## Basic vocabulary with mapping from indexes to words

In [2]:
SOS_token = 0 # Start of sentence
EOS_token = 1   # End of sentence

index2words = {
    SOS_token : "SOS",
    EOS_token : "EOS"
}

words = "How are you doing ? I am doing good and you ?"
words_list = set(words.lower().split(" "))
for word in words_list:
    index2words[len(index2words)] = word
    
print(index2words)

{0: 'SOS', 1: 'EOS', 2: 'good', 3: '?', 4: 'are', 5: 'am', 6: 'how', 7: 'i', 8: 'you', 9: 'doing', 10: 'and'}


In [3]:
# Getting the reverse mapping
words2index = {v: k for k, v in index2words.items()}
words2index

{'SOS': 0,
 'EOS': 1,
 'good': 2,
 '?': 3,
 'are': 4,
 'am': 5,
 'how': 6,
 'i': 7,
 'you': 8,
 'doing': 9,
 'and': 10}

In [5]:
# Functions to convert sentence to tensor
import torch

def convert2tensors(sentecnce):
    words_list = sentecnce.lower().split(" ")  # Convert the whole sentence into list of words
    indexes = [words2index[word] for word in words_list]  # Convert the words into indexes
    #print(indexes)
    return torch.tensor(indexes, dtype=torch.long).view(-1, 1)  # Convert the indexes into tensor

sentence = "How are you doing ?"
print(convert2tensors(sentence))

[6, 4, 8, 9, 3]
tensor([[6],
        [4],
        [8],
        [9],
        [3]])
